In [ ]:
#importing needed libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns


In [ ]:
#to change with the desired dataset
df = pd.read_csv("SBUX.csv")



In [ ]:
"""
------ Data preparation ------
"""

#converting dates in yyyy-mm-dd format
df['Date'] = pd.to_datetime(df['Date'])


#renamed the column for a pure personal preference
df.rename(columns={'Close/Last': 'Close'}, inplace=True)

#converted to purely numeric values
df['Close'] = df['Close'].replace('[\$,]', '', regex=True).astype(float)
df['Open'] = df['Open'].replace('[\$,]', '', regex=True).astype(float)
df['High'] = df['High'].replace('[\$,]', '', regex=True).astype(float)
df['Low'] = df['Low'].replace('[\$,]', '', regex=True).astype(float)

df = df.ffill()

# **Candle Stick**

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])


fig.update_layout(title='Candlestick Chart',
                  xaxis_title='Date',
                  yaxis_title='Price')

# **Scatter Plot**

In [ ]:
#converted to purely numeric values
df['Close'] = df['Close'].replace('[\$,]', '', regex=True).astype(float)
fig = px.scatter(df, x="Date", y="Close", title="Closing price over time")
fig.update_layout(yaxis_title='Price')

fig.show()

# **Volume bars for trading volumes**

In [ ]:
fig = px.histogram(df, x='Date', y='Volume', title='Trading Volume over Time')

fig.update_layout(xaxis_title='Date', yaxis_title='Volume')

fig.show()

# **Time Series**

In [ ]:
fig = go.Figure([go.Scatter(x=df['Date'], y=df['High'])])
fig.update_layout(title='Time Series',
                  xaxis_title='Date',
                  yaxis_title='Price')

# **Feature Engineering**

In [ ]:
from datetime import datetime, date, timedelta
Buy_date="10/21/2021"
span=100
targetdate=datetime.strptime(Buy_date,"%m/%d/%Y")


# **Rapporti Di Ritorno Cumulativi Target:3g,7g,14g**

In [ ]:
targetdate_3g=targetdate+timedelta(days=3)
targetdate_7g=targetdate+timedelta(days=7)
targetdate_14g=targetdate+timedelta(days=14)

df_filtered_base = df[df['Date'] == pd.to_datetime(targetdate)]
df_filtered_3g = df[df['Date'] == pd.to_datetime(targetdate_3g)]
df_filtered_7g = df[df['Date'] == pd.to_datetime(targetdate_7g)]
df_filtered_14g = df[df['Date'] == pd.to_datetime(targetdate_14g)]

if df_filtered_base.empty:
    print("No data for targetdate")
else:

    close_base = df_filtered_base['Close'].values[0]
    if df_filtered_3g.empty:
        print("Nessun dato per Ritorno cumulato a 3 giorni")
    else:
        close_3g = df_filtered_3g['Close'].values[0]
        print("Ritorno cumulato a 3 giorni:{:.2f}% ".format(((close_3g-close_base)/close_base)*100))
    if df_filtered_7g.empty:
        print("Nessun dato per Ritorno cumulato a 7 giorni")
    else:
        close_7g = df_filtered_7g['Close'].values[0]
        print("Ritorno cumulato a 7 giorni:{:.2f}% ".format(((close_7g-close_base)/close_base)*100))
    if df_filtered_14g.empty:
        print("Nessun dato per Ritorno cumulato a 14 giorni")
    else:
        close_14g = df_filtered_14g['Close'].values[0]
        print("Ritorno cumulato a 14 giorni:{:.2f}%".format(((close_14g-close_base)/close_base)*100))





# **Studio sui rapporti di ritorno:**

In [ ]:

df_ROI = pd.DataFrame()


df_100g = df[df['Date'] >= pd.to_datetime(targetdate)]
df_100g = df_100g[df_100g['Date'] <= pd.to_datetime(targetdate) + timedelta(days=100)]

Buy_date_close=df[df['Date'] == pd.to_datetime(targetdate)]['Close'].values[0]

for date in df_100g.iterrows():
    close = date[1]['Close']
    ROI= (close/Buy_date_close)*100
    df_ROI=pd.concat([df_ROI,pd.DataFrame({'Date': [date[1]['Date']], 'ROI': [ROI]})])
    df_ROI['ROI_Moving_Avg'] = df_ROI['ROI'].rolling(window=3).mean()
    Volatility = df_ROI['ROI'].std()

fig = px.line(df_ROI, x="Date", y=df_ROI.columns[:-1], title='ROI over time')
fig.update_layout(yaxis_title='ROI')
fig.show()
print("Volatility: {:.2f}%".format(Volatility))
print (df_ROI)


# **Indicatori tecnici:**

In [198]:
df_ema = pd.DataFrame()
df_ema['Date'] = df['Date']
df_ema['Close'] = df['Close']
df_ema['EMA'] = df['Close'].ewm(span=100, adjust=False).mean()
df_ema['RSI'] = ta.momentum.rsi(df['Close'])

fig = px.line(df_ema, x="Date", y=df_ema.columns, title='EMA over time')
fig.update_layout(yaxis_title='Value')
fig.show()
print (df_ema)

NameError: name 'ta' is not defined